# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [5]:
from openai import OpenAI

client = OpenAI(api_key="sk-MLOkqReWfMXcorYNEfA4321bA32a4c6e87FbA00839533a0a",base_url="https://api.xiaoai.plus/v1")

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示的是一个美丽的户外风景，背景是蓝天白云和一些轻微的云彩。图中有一条木制的栈道穿梭在一片繁茂的绿色草地中，引导观看者的视线深入到远处的景色。草地上生长着高高的草和一些灌木丛，这为画面增添了丰富的绿色和自然的生机。整个场景给人一种宁静和平和的感觉，是很典型的自然保护区或乡村公园的风景。这样的环境适合散步、放松心情，并吸引自然爱好者和摄影师。', role='assistant', function_call=None, tool_calls=None))


In [6]:
response.choices[0].message.content

'这幅图展示的是一个美丽的户外风景，背景是蓝天白云和一些轻微的云彩。图中有一条木制的栈道穿梭在一片繁茂的绿色草地中，引导观看者的视线深入到远处的景色。草地上生长着高高的草和一些灌木丛，这为画面增添了丰富的绿色和自然的生机。整个场景给人一种宁静和平和的感觉，是很典型的自然保护区或乡村公园的风景。这样的环境适合散步、放松心情，并吸引自然爱好者和摄影师。'

### 封装成一个函数 query_image_description

In [7]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(api_key="sk-MLOkqReWfMXcorYNEfA4321bA32a4c6e87FbA00839533a0a",base_url="https://api.xiaoai.plus/v1")  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [8]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图主要呈现了两只狗的幽默对比。左侧的狗被过度P图成一个拥有巨大肌肉的人形，代表了“16岁的我，工作后的我”。右侧则是一只普通的柴犬，被描绘成憨态可掬的样子，下方配文表达了对工作或日常生活带来忧虑和疲惫的感受，与左侧肌肉狗的自信形象形成鲜明对比。

整体而言，这是一种常见的搞笑风格的对比图，通过夸张和反差来产生幽默效果，表达了很多人对年轻时期与成年后现实之间差距的感慨。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [14]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(api_key="sk-MLOkqReWfMXcorYNEfA4321bA32a4c6e87FbA00839533a0a",base_url="https://api.xiaoai.plus/v1")  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [15]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图显示了1980年到2020年美国、中国、日本和德国的国内生产总值（GDP）的比较。图中的y轴代表国内生产总值（以万亿美元为单位），x轴代表年份。

从图中可以看出以下几点：
- 美国的GDP（蓝线）在这四个国家中始终处于最高位置，并且呈现稳定增长的趋势。
- 中国的GDP（红线）从1990年代初期开始显著增长，尤其从2000年以后增长迅速，到2020年接近美国的GDP。
- 日本的GDP（紫线）增长相对平稳，但在1990年代末期到2000年代初期有所下降，此后基本保持稳定。
- 德国的GDP（绿线）增长较为缓慢且稳定，没有太大波动。

这张图有效地展示了过去四十年这四个经济大国的经济增长轨迹和相对表现。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [16]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片显示的是一本笔记本上的笔记，内容主要涉及机器学习模型调优的技术，特别是在自然语言处理领域的应用。笔记内容具体如下：

1. **Prompt Tuning（提示调优）** - 这是一种用于小型模型的技术，通过仅调整模型输入的一部分（称为“prompt”），达到提升模型性能的目的。笔记中写到X代表输入的Embedding向量，而Y则是通过权重矩阵W转换X后的输出。

2. **Prefix Tuning（前缀调优）** - 这种方法涉及在模型的输入前添加一些固定的前缀，这些前缀经过训练可以改善模型的表现。这里的W'代表添加了前缀的新权重矩阵，该技术通常用于Transformer类模型的Encoder/Decoder结构。

3. **LoRA（低秩适应）** - 这是另一种模型调优方法，通过修改模型中权重矩阵的子矩阵来实现参数的低秩更新，从而节省计算资源并保持模型表现。公式中的ΔW = AB表示权重的修改，其中A和B是较小的矩阵，提供了参数调整的自由度。

笔记还提到了一些实现细节，例如Q LoRA和与LoRA相关的存储需求（如LAMA与Q LoRA的存储需求对比），这表明了LoRA方法在调优时对资源的节约。

总的来说，这张图片的内容吸收了一些先进的技术和方法，用于提升自然语言处理模型在特定任务上的表现，同时注意到计算资源的优化配置。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [17]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片显示的是一本笔记本上的笔记，内容主要涉及机器学习模型调优的技术，特别是在自然语言处理领域的应用。笔记内容具体如下：

1. **Prompt Tuning（提示调优）** - 这是一种用于小型模型的技术，通过仅调整模型输入的一部分（称为“prompt”），达到提升模型性能的目的。笔记中写到X代表输入的Embedding向量，而Y则是通过权重矩阵W转换X后的输出。

2. **Prefix Tuning（前缀调优）** - 这种方法涉及在模型的输入前添加一些固定的前缀，这些前缀经过训练可以改善模型的表现。这里的W'代表添加了前缀的新权重矩阵，该技术通常用于Transformer类模型的Encoder/Decoder结构。

3. **LoRA（低秩适应）** - 这是另一种模型调优方法，通过修改模型中权重矩阵的子矩阵来实现参数的低秩更新，从而节省计算资源并保持模型表现。公式中的ΔW = AB表示权重的修改，其中A和B是较小的矩阵，提供了参数调整的自由度。

笔记还提到了一些实现细节，例如Q LoRA和与LoRA相关的存储需求（如LAMA与Q LoRA的存储需求对比），这表明了LoRA方法在调优时对资源的节约。

总的来说，这张图片的内容吸收了一些先进的技术和方法，用于提升自然语言处理模型在特定任务上的表现，同时注意到计算资源的优化配置。

![](./images/handwriting_1.jpg)

In [19]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片显示的是一本笔记本的两页，上面记录了一些关于机器学习、特别是自然语言处理（NLP）和深度学习模型中的技术和方法的手写笔记。笔记中提及了多种技术和概念，主要聚焦在“Transformers”（变压器模型），这是一种广泛用于NLP领域的深度学习模型架构。

左页内容：
- 提到了“H/F Transformers”和“标签”，可能是在讨论不同类型的Transformers模型或是它们的用途。
- 提到了benchmark（基准测试），以及一些优化Transformers模型的方法，如“PEFT”、“SOTA”（State Of The Art，最先进的技术）和“PBFT Methods”。
- 进一步详细介绍了各种类型的tuning（调优）技术，如“Prompt Tuning”、“Adapter”、“Prefix”，“P-Tuning V1”和V2、“Soft Prompts”以及“Hand Prompts”。这些都是用于精细调整预训练模型以适应特定任务而不需重训练整个模型的技术。

右页内容：
- 更加专注于“multi-modality Instruction Ft. LLMS”和各种新的调整技术，如“LORA”，“LoRA”，“QLoRA”和“AdapLoRA”。
- 深入探讨了“prefix-tuning & Adapters”的概念，并对其中的细节进行了一些批注或补充，例如如何通过改进的方法在大规模语言模型（LLMs）中实现更好的性能。
- 讨论了如何通过插入、修改代表性结构以及使用比较函数等技术优化模型的能力。
- 提到了“MAM Adapters”和对于提升模型性能的不同策略的讨论。

总体而言，这些笔记可能是某个研究人员或学生在学习或工作中记录的，涉及NLP和深度学习领域中最新的研究进展和技术细节。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [22]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="图里是AI培训课程的作业，描述下作业内容以及目前完成情况", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"


content = query_base64_image_description("./images/ming test.png")
display(Markdown(content))

图片中展示的是一份AI培训课程的书面作业，包括了以下几个任务：

1. 翻译（translator）
2. 销售机器人（sale-bot）
3. 使用字符（use character）
4. 历史中的人工智能（AI in history）
5. 故事内容（story content）
6. 网页设计（4 UV）
7. 写下你的体会（markdown）

目前作业完成情况为：

- "翻译"任务已完成（打勾）
- "销售机器人"任务已完成（打勾）
- "使用字符"任务已完成（打勾）
- "历史中的人工智能"任务已完成（打勾）

剩余的任务还未完成。